# Analyse en Composantes Principales (ACP) - Décathlon

Analyse des profils d'athlètes de décathlon pour identifier des groupes similaires.

## Importation des biblio

In [12]:
#import des librairies nécessaires à notre analyse
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

#config visuelle
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

## Chargement des données

In [13]:
#chargement du CSV
df = pd.read_csv('decathlon.csv', sep=';')

#aperçu
df.head()

,Nom,100m,Longueur,Poids,Hauteur,400m,110m H,Disque,Perche,Javelot,1500m,Classement,Points,Competition
0,Sebrle,10.85,7.84,16.36,2.12,48.36,14.05,48.72,5.0,70.52,280.01,1,8893,JO
1,Clay,10.44,7.96,15.23,2.06,49.19,14.13,50.11,4.9,69.71,282.00,2,8820,JO
2,Karpov,10.50,7.81,15.93,2.09,46.81,13.97,51.65,4.6,55.54,278.11,3,8725,JO
3,Macey,10.89,7.47,15.73,2.15,48.97,14.56,48.34,4.4,58.46,265.42,4,8414,JO
4,Warners,10.62,7.74,14.48,1.97,47.97,14.01,43.73,4.9,55.39,278.05,5,8343,JO


## Exploration des données

In [14]:
#infos sur la data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Nom          41 non-null     object 
 1   100m         41 non-null     float64
 2   Longueur     41 non-null     float64
 3   Poids        41 non-null     float64
 4   Hauteur      41 non-null     float64
 5   400m         41 non-null     float64
 6   110m H       41 non-null     float64
 7   Disque       41 non-null     float64
 8   Perche       41 non-null     float64
 9   Javelot      41 non-null     float64
 10  1500m        41 non-null     float64
 11  Classement   41 non-null     int64  
 12  Points       41 non-null     int64  
 13  Competition  41 non-null     object 
dtypes: float64(10), int64(2), object(2)
memory usage: 4.6+ KB


In [15]:
#stats descriptives
df.describe()

,100m,Longueur,Poids,Hauteur,400m,110m H,Disque,Perche,Javelot,1500m,Classement,Points
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,10.998049,7.260000,14.477073,1.976829,49.616341,14.605854,44.325610,4.762439,58.316585,279.024878,12.121951,8005.365854
std,0.263023,0.316402,0.824428,0.088951,1.153451,0.471789,3.377845,0.278000,4.826820,11.673247,7.918949,342.385145
min,10.440000,6.610000,12.680000,1.850000,46.810000,13.970000,37.920000,4.200000,50.310000,262.100000,1.000000,7313.000000
25%,10.850000,7.030000,13.880000,1.920000,48.930000,14.210000,41.900000,4.500000,55.270000,271.020000,6.000000,7802.000000
50%,10.980000,7.300000,14.570000,1.950000,49.400000,14.480000,44.410000,4.800000,58.360000,278.050000,11.000000,8021.000000
75%,11.140000,7.480000,14.970000,2.040000,50.300000,14.980000,46.070000,4.920000,60.890000,285.100000,18.000000,8122.000000
max,11.640000,7.960000,16.360000,2.150000,53.200000,15.670000,51.650000,5.400000,70.520000,317.000000,28.000000,8893.000000


## Préparation des données pour l'ACP

In [16]:
#on vire les colonnes qui nous intéressent pas pour l'analyse
colonnes_a_exclure = ['Competition', 'Points', 'Classement']
df_acp = df.drop(columns=colonnes_a_exclure)

#on met les noms des athlètes de côté
# mais on index pour après pouvoir les retrouver
noms_athletes = df_acp['Nom']
df_acp = df_acp.set_index('Nom')

#aperçu avant acp, vu que c'est Joyce qui nous fournit les données
# on saute l'ETL
df_acp.head()

,100m,Longueur,Poids,Hauteur,400m,110m H,Disque,Perche,Javelot,1500m
Nom,,,,,,,,,,
Sebrle,10.85,7.84,16.36,2.12,48.36,14.05,48.72,5.0,70.52,280.01
Clay,10.44,7.96,15.23,2.06,49.19,14.13,50.11,4.9,69.71,282.00
Karpov,10.50,7.81,15.93,2.09,46.81,13.97,51.65,4.6,55.54,278.11
Macey,10.89,7.47,15.73,2.15,48.97,14.56,48.34,4.4,58.46,265.42
Warners,10.62,7.74,14.48,1.97,47.97,14.01,43.73,4.9,55.39,278.05


## Vérification des valeurs manquantes

In [17]:
#check rapide pour voir si Joyce a bien fait son taf
df_acp.isnull().sum()

100m        0
Longueur    0
Poids       0
Hauteur     0
400m        0
110m H      0
Disque      0
Perche      0
Javelot     0
1500m       0
dtype: int64

## Standardisation des données